In [ ]:
from IPython.display import HTML
HTML(open('../style.css').read())

In [ ]:
%load_ext nb_mypy

In [ ]:
Variable = str
Literal  = Variable | tuple[str, Variable]
Clause   = frozenset[Literal]
CNF      = set[Clause]

# Refutational Completeness of the Cut Rule

This notebook implements a number of procedures that are needed in our proof of the <em style="color:blue">refutational completeness</em> of the cut rule.

The function $\texttt{complement}(l)$ computes the *complement* of a literal $l$.
As we are only dealing with formulas in CNF that are written in set notation and these formulas do not contain $\top$ or $\bot$, a literal is either 
a propositional variable or the negation of a propositional variable.
Therefore, if $p$ is a propositional variable, we have the following: 
* $\texttt{complement}(p) = \neg p$,
* $\texttt{complement}(\neg p) = p$.

In [ ]:
def complement(l: Literal) -> Literal:
    'Compute the complement of the literal l.'
    match l:
        case ('¬', p): return p 
        case p       : return ('¬', p) # type: ignore
    return None # type: ignore

In [ ]:
complement('p')

In [ ]:
complement(('¬', 'p'))

The function $\texttt{extractVariable}(l)$ extracts the propositional variable from the literal $l$.
If $p$ is a propositional variable, we have the following: 
<ol>
    <li>$\texttt{extractVariable}(p) = p$,
    </li>
    <li>$\texttt{extractVariable}(\neg p) = p$.
    </li>
</ol>

In [ ]:
def extractVariable(l: Literal) -> Variable:
    'Extract the variable from the literal l.'
    match l:
        case ('¬', p): return p
        case p       : return p # type: ignore          
    return None # type: ignore

In [ ]:
extractVariable('p')

In [ ]:
extractVariable(('¬', 'q'))

The function $\texttt{collectsVariables}(M)$ takes a set of clauses $M$ as its input and computes the set of all propositional variables occurring in $M$.  The clauses in $M$ are represented as sets of literals.

In [ ]:
def collectVariables(M: set[Clause]) -> set[Variable]:
    "Return the set of all variables occurring in clauses of M."
    return { extractVariable(l) for Clause in M for l in Clause }

In [ ]:
C1 = frozenset({ 'p', 'q', 'r' })
C2 = frozenset({ ('¬', 'p'), ('¬', 'q'), ('¬', 's') })
collectVariables({ C1, C2 })

Given two clauses $C_1$ and $C_2$ that are represented as sets of literals, the function `cutRule`$(C_1, C_2)$ computes the set of all clauses that can be derived from $C_1$ and $C_2$ using the *cut rule*.  In set notation, the cut rule is the following rule of inference:
$$
   \frac{\displaystyle \;C_1\cup \{l\} \quad C_2 \cup \bigl\{\overline{\,l\,}\bigr\}}{\displaystyle C_1 \cup C_2}
$$

In [ ]:
def cutRule(C1: Clause, C2: Clause) -> set[Clause]:
    "Return the set of all clauses that can be deduced by the cut rule from C1 and C2."
    return { C1 - {l} | C2 - {complement(l)} for l in C1
                                             if  complement(l) in C2
           }

In [ ]:
C1 = frozenset({ ('¬', 'q'), ('¬', 'r') })
C2 = frozenset({ 'q', 'r' })
cutRule(C1, C2)

In the expression `saturate(Clauses)` below, `Clauses` is a set of *clauses*, where each clause is a set of *literals*.  The call `saturate(Clauses)` computes the set of all clauses that can be derived from clauses in the set `Clauses` using the *cut rule*.  The function keeps applying the cut rule until either no new clauses can be derived, or the empty clause $\{\}$ is derived.  The resulting set of clauses is *saturated* in the following sense:  If $C_1$ and $C_2$ are clauses from the set `Clauses` and the clause $D$ can be derived from $C_1$ and $C_2$ via the cut rule, then $D \in \texttt{Clauses}$ or $\{\} \in \texttt{Clauses}$.

In [ ]:
def saturate(Clauses: set[Clause]) -> set[Clause]:
    while True:
        Derived = { C for C1 in Clauses
                      for C2 in Clauses
                      for C  in cutRule(C1, C2)
                  }
        if frozenset() in Derived:
            return { frozenset() }  # This is the set notation of ⊥ in CNF.
        Derived -= Clauses          # Remove clauses that were already present before.
        if Derived == set():        # No new clauses have been found.
            return Clauses
        Clauses |= Derived

In [ ]:
C1 = frozenset({ 'p', 'q' })
C2 = frozenset({ ('¬', 'p') })
C3 = frozenset({ ('¬', 'p'), ('¬', 'q') })
saturate({C1, C2, C3})

The function $\texttt{findValuation}(\texttt{Clauses})$ takes a set of clauses as input.  The function tries to compute a variable interpretation that makes all of these clauses `True`.  If this attempt is successful, a set of literals is returned.  This set of literals does not contain  any complementary literals and therefore corresponds to a variable assignment satisfying all clauses.  If $\texttt{Clauses}$ is unsatisfiable, `False` is returned.

In [ ]:
def findValuation(Clauses: set[Clause]) -> bool | set[Literal]:
    "Given a set of Clauses, find a propositional valuation satisfying all of these clauses."
    Variables = collectVariables(Clauses)
    Clauses   = saturate(Clauses)
    if frozenset() in Clauses:  # The set Clauses is inconsistent.
        return False
    Literals: set[Literal] = set()
    for p in Variables:
        if any(C for C in Clauses 
                 if  p in C and C - {p} <= { complement(l) for l in Literals }
              ):
            Literals |= { p }
        else:
            Literals |= { ('¬', p) }
    return Literals

In [ ]:
C01: Clause = frozenset({ 'r', 'p', 's' })
C02: Clause = frozenset({ 'r', 's' })
C03: Clause = frozenset({ 'p', 'q', 's' })
C04: Clause = frozenset({ ('¬', 'p'), ('¬', 'q') })
C05: Clause = frozenset({ ('¬', 'p'), 's', ('¬', 'r') })
C06: Clause = frozenset({ 'p', ('¬', 'q'), 'r'})
C07: Clause = frozenset({ ('¬', 'r'), ('¬', 's'), 'q' })
C08: Clause = frozenset({ ('¬', 'p'), ('¬', 's')})
C09: Clause = frozenset({ 'p', ('¬', 'r'), ('¬', 'q') })
C10: Clause = frozenset({ ('¬', 'p'), 'r', 'q', ('¬', 's') })
C11: Clause = frozenset({ 'p', 'q', 'r', ('¬', 's') })
Clauses: set[Clause] = { C01, C02, C03, C04, C05, C06, C07, C08, C09, C10 }
findValuation(Clauses)

In [ ]:
Clauses: set[Clause] = { C01, C02, C03, C04, C05, C06, C07, C08, C09, C10, C11 }
findValuation(Clauses)

In [ ]:
saturate(Clauses)